Import the required libraries and modules that you would need.


In [1]:
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.preprocessing import StandardScaler


pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')

Read that data into Python and call the dataframe churnData.


In [2]:
churndata = pd.read_csv('Customer-Churn.csv')
churndata.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [3]:
churndata.shape

(7043, 16)

Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.


In [4]:
churndata.dtypes


gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [5]:
churndata.loc[[488]]


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,Yes,No,Yes,Yes,Yes,No,Two year,52.55,,No


In [6]:
churndata['totalcharges'] = pd.to_numeric(churndata['TotalCharges'], errors='coerce')


In [7]:
churndata.dtypes


gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
totalcharges        float64
dtype: object

Check for null values in the dataframe. Replace the null values.


In [8]:
churndata.isnull().values.sum()

11

In [9]:

# creating bool series True for NaN values 
bool_series = pd.isnull(churndata["totalcharges"]) 
    
# filtering data 
# displaying data only with Gender = NaN 
churndata[bool_series] 

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn,totalcharges
488,Female,0,Yes,Yes,0,No,Yes,No,Yes,Yes,Yes,No,Two year,52.55,,No,NaN
753,Male,0,No,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,20.25,,No,NaN
936,Female,0,Yes,Yes,0,Yes,Yes,Yes,Yes,No,Yes,Yes,Two year,80.85,,No,NaN
1082,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,25.75,,No,NaN
1340,Female,0,Yes,Yes,0,No,Yes,Yes,Yes,Yes,Yes,No,Two year,56.05,,No,NaN
3331,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,19.85,,No,NaN
3826,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,25.35,,No,NaN
4380,Female,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,20.00,,No,NaN
5218,Male,0,Yes,Yes,0,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,19.70,,No,NaN
6670,Female,0,Yes,Yes,0,Yes,No,Yes,Yes,Yes,Yes,No,Two year,73.35,,No,NaN


by checking out the original values on the dataframe for the totalcharges columns i've decided to fill in the nulls with 0, seems like tenure multiplied by monthly fees is how you calculate the total charge and all of this rows have 0 tenure hence why they havent been charged yet.

In [10]:
churndata['totalcharges'] = churndata['totalcharges'].fillna(0)


In [11]:
churndata.isnull().values.sum()

0

In [12]:
churndata1= churndata

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:
Scale the features either by using normalizer or a standard scaler.
Split the data into a training set and a test set.
Fit a logistic regression model on the training data.
Check the accuracy on the test data.

In [13]:
churndata.select_dtypes(np.number)


,SeniorCitizen,tenure,MonthlyCharges,totalcharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
...,...,...,...,...
7038,0,24,84.80,1990.50
7039,0,72,103.20,7362.90
7040,0,11,29.60,346.45
7041,1,4,74.40,306.60


In [14]:
y = churndata['Churn']
# The Features
numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'totalcharges']
numericals = churndata[numericals]

In [15]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(7043, 4)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)


In [17]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.82      0.92      0.87      1539
         Yes       0.68      0.45      0.54       574

    accuracy                           0.79      2113
   macro avg       0.75      0.69      0.70      2113
weighted avg       0.78      0.79      0.78      2113



In [18]:
classification.score(X_test, y_test)


0.7941315664931378

Managing imbalance in the dataset

Check for the imbalance.
Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
Each time fit the model and see how the accuracy of the model is.

In [19]:
y.value_counts()


No     5174
Yes    1869
Name: Churn, dtype: int64

In [20]:
####upsampling


In [21]:
churndata.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'totalcharges'],
      dtype='object')

In [22]:
category_no = churndata[churndata['Churn'] == 'No']
category_yes = churndata[churndata['Churn'] == 'Yes']

# Upsampling 
# As we are going to repeat observations, the random samples can be picked more then once,
# threfore we need to use the keyword: replace=True

category_yes = category_yes.sample(len(category_no), replace=True)  
(category_yes.shape)



(5174, 17)

In [23]:
data3 = pd.concat([category_no, category_yes], axis=0)
#shuffling the data
data3 = data3.sample(frac=1)
print(data3['Churn'].value_counts())

No     5174
Yes    5174
Name: Churn, dtype: int64


In [24]:
data3

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn,totalcharges
1764,Male,0,Yes,Yes,63,Yes,Yes,No,Yes,Yes,No,Yes,Two year,75.70,4676.7,No,4676.70
3905,Female,0,No,No,3,No,No,No,No,No,Yes,No,Month-to-month,35.20,108.95,Yes,108.95
4357,Female,0,Yes,No,15,Yes,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,20.45,330.8,No,330.80
4978,Female,0,No,No,1,Yes,No,No,No,No,No,No,Month-to-month,44.10,44.1,Yes,44.10
4334,Female,0,No,No,11,Yes,Yes,No,No,No,No,No,Month-to-month,76.40,838.7,No,838.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,Male,0,No,No,1,No,No,No,No,No,No,No,Month-to-month,24.80,24.8,Yes,24.80
1809,Male,1,Yes,No,32,Yes,No,No,No,No,No,Yes,Month-to-month,85.00,2642.05,Yes,2642.05
983,Male,0,No,No,1,Yes,No,No,No,No,No,Yes,Month-to-month,86.60,86.6,Yes,86.60
996,Female,0,No,No,34,Yes,No,No,No,No,No,No,Month-to-month,70.00,2416.1,Yes,2416.10


In [27]:
y = data3['Churn']
# The Features
numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'totalcharges']
numericals = data3[numericals]

In [28]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(10348, 4)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)


In [30]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.72      0.75      0.74      1523
         Yes       0.75      0.72      0.74      1582

    accuracy                           0.74      3105
   macro avg       0.74      0.74      0.74      3105
weighted avg       0.74      0.74      0.74      3105



In [31]:
classification.score(X_test, y_test)


0.7352657004830918

Upsampling is making the model have lower score but higher precision calculating yes 

In [32]:
###now downsampling

In [33]:
cat_no = churndata1[churndata1['Churn'] == 'No']
cat_yes = churndata1[churndata1['Churn'] == 'Yes']


cat_no = cat_no.sample(len(cat_yes))
print(cat_no.shape)
print(cat_yes.shape)

data2 = pd.concat([cat_no, cat_yes], axis=0)
#shuffling the data
data2 = data2.sample(frac=1)
data2['Churn'].value_counts()

(1869, 17)
(1869, 17)


No     1869
Yes    1869
Name: Churn, dtype: int64

In [34]:
data4 = pd.concat([cat_no, cat_yes], axis=0)
#shuffling the data
data3 = data3.sample(frac=1)
print(data4['Churn'].value_counts())

No     1869
Yes    1869
Name: Churn, dtype: int64


In [37]:
data4.isnull().values.sum()

0

In [38]:
y = data4['Churn']
# The Features
numericals = ['tenure', 'SeniorCitizen', 'MonthlyCharges', 'totalcharges']
numericals = data4[numericals]

In [39]:
transformer = StandardScaler().fit(numericals)
X_stan = transformer.transform(numericals)
X_stan.shape

(3738, 4)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_stan, y, test_size=0.3, random_state=42)


In [41]:
classification = LogisticRegression(random_state=42, max_iter=10000)
classification.fit(X_train, y_train)
predictions = classification.predict(X_test)
print(metrics.classification_report(y_test, predictions))

              precision    recall  f1-score   support

          No       0.75      0.74      0.75       578
         Yes       0.73      0.74      0.74       544

    accuracy                           0.74      1122
   macro avg       0.74      0.74      0.74      1122
weighted avg       0.74      0.74      0.74      1122



In [42]:
classification.score(X_test, y_test)


0.7406417112299465

model behaves more or less the same